Mon titre

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
#import sys
#import pprint
#import pylab
from mpl_toolkits.mplot3d import Axes3D
#import random

In [69]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook

In [70]:
conn_string = "host='pgslave' port=5433 dbname='refgeotest' user='pgadmin' password='pgadmin'"

In [71]:
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print "Connected!\n"

Connected!



In [72]:
id_releve  = 'L97_2011'

In [73]:
z_factor = 3

In [74]:
point_size = 1

In [75]:
max_points = 120000

In [76]:
sql = 'select distinct id_releve, st_x(geoml93) x, st_y(geoml93) as y, (-profondeur::real*'+ str(z_factor) +'::real)::real as p  from bathy_profondeurs where  id_releve = \''+ id_releve +'\' limit '+ str(max_points) +' ;'

In [77]:
sql

"select distinct id_releve, st_x(geoml93) x, st_y(geoml93) as y, (-profondeur::real*3::real)::real as p  from bathy_profondeurs where  id_releve = 'L97_2011' limit 120000 ;"

In [78]:
#old way :
#cursor.execute(sql)
#rows = cursor.fetchall()
#df = pd.DataFrame( [[ij for ij in i] for i in rows] )
#df.rename(columns={0: 'id_releve', 1: 'x', 2: 'y', 3: 'prof'}, inplace=True);


In [79]:
df = pd.read_sql(sql, conn)

In [80]:
df.head()

,id_releve,x,y,p
0,L97_2011,644963.728244,6.476987e+06,-40.29
1,L97_2011,644775.743094,6.478396e+06,-6.09
2,L97_2011,644666.483252,6.476737e+06,-14.25
3,L97_2011,644760.737287,6.478410e+06,-8.76
4,L97_2011,644917.314176,6.475949e+06,-6.09


In [81]:
n_points = df['p'].count()

In [82]:
n_points

13953

In [83]:
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('regis.haubourg', 'qyh9ssvz4u')

# import pour sortir en fichier html 
from plotly.offline.offline import _plot_html


In [84]:
x=df['x']
y=df['y']
z=df['p'] * z_factor

In [85]:


trace1 = Scatter3d(
     x=x,
     y=y,
     z=z,
     mode='markers',
     marker=dict(
            size=point_size,
            symbol='circle',
            color=z,                # set color to an array/list of desired values
            colorscale='Jet',   # choose a colorscale
            opacity=0.9  ),
    stream=dict(maxpoints=1000)
)    
layout = Layout(        
     title = 'Bathymetrie '+ id_releve,
     scene=Scene(
        xaxis=XAxis(title='x'),
        yaxis=YAxis(title='y'),
        zaxis=ZAxis(title='z prof'),
        aspectmode = "data"
    )
)
data = Data([trace1])
fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='bathymetrie '+ id_releve)

C:\OSGEO4~1\apps\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [95]:
plot( { 'data' : [
Scatter3d(
     x=x,
     y=y,
     z=z,
     mode='markers',
     marker=dict(
            size=point_size,
            symbol='circle',
            color=z,                # set color to an array/list of desired values
            colorscale='Jet',   # choose a colorscale
            opacity=0.9  ),
    stream=dict(maxpoints=1000)
)    ], 
       
'layout' : Layout(        
     title = 'Bathymetrie '+ id_releve,
     scene=Scene(
        xaxis=XAxis(title='x'),
        yaxis=YAxis(title='y'),
        zaxis=ZAxis(title='z prof'),
        aspectmode = "data"
    ))
}
    , filename='bathymetrie_'+ id_releve
         )

C:\OSGEO4~1\apps\Python27\lib\site-packages\plotly\offline\offline.py:385: UserWarning:

Your filename `bathymetrie_L97_2011` didn't end with .html. Adding .html to the end of your file.



'file://F:\\Outils\\QGISDEV\\SCRIPTS_PYTHON\\bathymetrie_L97_2011.html'

In [ ]:
cursor.close()
conn.close()